In [1]:
from transformers import PreTrainedTokenizerFast
from models import ETPOL
import pandas as pd
from pathlib import Path


context_length = 512
d_model = 768
num_heads = 16
num_hidden_layers = 4
d_hidden = 2048
num_encoders = 7 
 
 

lr =  6.871199163253075e-05
pdrop = 0.3099236390195026


model = ETPOL(
    vocab_size=20000,
    context_length=context_length,
    d_model=d_model,
    d_hidden=d_hidden,
    num_hidden_layers=num_hidden_layers,
    num_heads=num_heads,
    num_encoders=num_encoders,
    pdrop=pdrop 
)

tokenizer = PreTrainedTokenizerFast.from_pretrained(Path('wp_tokenizer'), model_max_lenth=context_length)


/home/fhaytonruffner25/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
model.load_state_dict(torch.load('./models/etpol.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [3]:

reverse_label_map = {1: 'left', 0: 'right'}
def get_pol(content):
    tokenized = tokenizer.encode(content,
                                 max_length=context_length,
                                 padding='max_length',
                                 truncation = True
                            )
    inp = torch.tensor(tokenized).unsqueeze(0)
    logits = model(inp)
    return reverse_label_map[int(torch.argmax(logits, dim=-1))]



In [7]:
counts = {
    'left': 0,
    'right': 0
}

model.eval()
for i in range(10):
    counts[get_pol("anarcho biden capitalism")] += 1
    
print(counts)


{'left': 0, 'right': 10}
